<a href="https://colab.research.google.com/github/johju/dl_lab1/blob/master/my_FMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Källor:
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_classification.ipynb
# https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/fashion_mnist.ipynb
# Kursboken Deep Learning with Python

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras import layers
from keras import models

print(tf.__version__)

1.13.1


Using TensorFlow backend.


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


4423680/4422102 [==============================] - 0s 0us/step


In [0]:

# Här en model utan några större justeringar men med dropout
# samt maxpooling för att downsampla parameters, dropout mellan dense layers för att tygla överträning
model_one = models.Sequential()
model_one.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_one.add(layers.MaxPooling2D((2, 2)))

model_one.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_one.add(layers.MaxPooling2D((2, 2)))

model_one.add(layers.Conv2D(128, (3, 3), activation='relu'))

model_one.add(layers.Flatten())
model_one.add(layers.Dense(128, activation='relu'))
model_one.add(layers.Dropout(0.5))
model_one.add(layers.Dense(10, activation='softmax'))

model_one.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_84 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten_39 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_77 (Dense)             (None, 128)               147584    
__________

In [0]:
model_one.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model_one.fit(train_images, train_labels, epochs=10, batch_size=256)

test_loss, test_acc = model_one.evaluate(test_images, test_labels)

print('model_one test accuracy:', test_acc)

Epoch 1/10
60000/60000 [==============================] - 10s 163us/step - loss: 0.7485 - acc: 0.7298
Epoch 2/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.4545 - acc: 0.8364
Epoch 3/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.3843 - acc: 0.8636
Epoch 4/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.3437 - acc: 0.8781
Epoch 5/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.3136 - acc: 0.8867
Epoch 6/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.2913 - acc: 0.8956
Epoch 7/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.2745 - acc: 0.9015
Epoch 8/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.2614 - acc: 0.9059
Epoch 9/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.2485 - acc: 0.9111
Epoch 10/10
10000/10000 [==============================] - 4s 399us/step
model_one test accuracy: 

In [0]:

# Den här modellen inspirerad ifrån:
# https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/fashion_mnist.ipynb

# https://towardsdatascience.com/dont-use-dropout-in-convolutional-networks-81486c823c16
# som rekommenderar BatchNormalization

# samt med padding från föreläsningen om padding, strides på kurssidan.
# Dropout mellan dense layers för att tygla överträning

model_two = models.Sequential()

model_two.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model_two.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model_two.add(layers.BatchNormalization())

model_two.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model_two.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model_two.add(layers.BatchNormalization())

model_two.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model_two.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model_two.add(layers.Flatten())
model_two.add(layers.Dense(128, activation='relu'))
model_two.add(layers.Dropout(0.5))
model_two.add(layers.Dense(10, activation='softmax'))

model_two.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 7, 7, 64)          256       
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 7, 7, 128)         73856     
__________

In [0]:

model_two.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model_two.fit(train_images, train_labels, epochs=6, batch_size=256)

test_loss, test_acc = model_two.evaluate(test_images, test_labels)

print('model_two test accuracy:', test_acc)


Epoch 1/6
60000/60000 [==============================] - 7s 120us/step - loss: 0.5960 - acc: 0.7935
Epoch 2/6
60000/60000 [==============================] - 5s 90us/step - loss: 0.3669 - acc: 0.8713
Epoch 3/6
60000/60000 [==============================] - 5s 90us/step - loss: 0.3090 - acc: 0.8905
Epoch 4/6
60000/60000 [==============================] - 5s 90us/step - loss: 0.2768 - acc: 0.9020
Epoch 5/6
60000/60000 [==============================] - 5s 90us/step - loss: 0.2498 - acc: 0.9116
Epoch 6/6
10000/10000 [==============================] - 2s 167us/step
model_two test accuracy: 0.9152
